In [1]:
import tensorflow as tf


In [2]:
def fixed_padding(inputs, kernel_size, data_format):
    """Pads the input along the spatial dimensions independently of input size.

    Args:
      inputs: A tensor of size [batch, channels, height_in, width_in] or
        [batch, height_in, width_in, channels] depending on data_format.
      kernel_size: The kernel to be used in the conv2d or max_pool2d operation.
                   Should be a positive integer.
      data_format: The input format ('channels_last' or 'channels_first').

    Returns:
      A tensor with the same format as the input with the data either intact
      (if kernel_size == 1) or padded (if kernel_size > 1).
    """
    pad_total = kernel_size - 1
    pad_beg = pad_total // 2
    pad_end = pad_total - pad_beg

    if data_format == 'channels_first':
        padded_inputs = tf.pad(inputs, [[0, 0], [0, 0],
                                        [pad_beg, pad_end], [pad_beg, pad_end]])
    else:
        padded_inputs = tf.pad(inputs, [[0, 0], [pad_beg, pad_end],
                                        [pad_beg, pad_end], [0, 0]])
    return padded_inputs

In [3]:
_BATCH_NORM_DECAY = 0.997
_BATCH_NORM_EPSILON = 1e-5
is_training=True
def batch_norm_relu(inputs, is_training, data_format):
    """Performs a batch normalization followed by a ReLU."""
    # We set fused=True for a significant performance boost. See
    # https://www.tensorflow.org/performance/performance_guide#common_fused_ops
    inputs = tf.layers.batch_normalization(
        inputs=inputs, axis=1 if data_format == 'channels_first' else 3,
        momentum=_BATCH_NORM_DECAY, epsilon=_BATCH_NORM_EPSILON, center=True,
         scale=True, training=is_training, fused=True)
    inputs = tf.nn.relu(inputs)
    return inputs

In [18]:
def conv_block(inputs,filters,strides,is_training,data_format,name_scope):
    with tf.name_scope(name_scope):
        #short_cut
        short_cut = inputs
        inputs = batch_norm_relu(inputs, is_training, data_format)
        
        short_cut = tf.layers.conv2d(inputs,filters=filters*4,kernel_size=(1,1),strides=(strides,strides),
                                     padding='valid',use_bias=False,data_format)
        
        #conv
        inputs = tf.layers.conv2d(inputs,filters=filters,kernel_size=(1,1),strides=(strides,strides),padding='valid')
        inputs = batch_norm_relu(inputs, is_training,data_format)

        inputs = tf.layers.conv2d(inputs,filters=filters,kernel_size=(3,3),strides=(1,1),padding='same')
        inputs = batch_norm_relu(inputs, is_training,data_format)

        inputs = tf.layers.conv2d(inputs,filters=filters*4, kernel_size=(1,1),strides=(1,1),padding='valid')
        inputs = batch_norm_relu(inputs, is_training,data_format)

        outputs = tf.add(inputs,short_cut,'out_put')
        return outputs

def identity_block(inputs,filters,strides,is_training,data_format,name_scope):
    with tf.name_scope(name_scope):
        #short_cut
        short_cut = inputs
       
        inputs = batch_norm_relu(inputs, is_training, data_format)
        #conv
        inputs = tf.layers.conv2d(inputs,filters=filters,kernel_size=(1,1),strides=(1,1),padding='valid')
        inputs = batch_norm_relu(inputs, is_training,data_format)

        inputs = tf.layers.conv2d(inputs,filters=filters,kernel_size=(3,3),strides=(1,1),padding='same')
        inputs = batch_norm_relu(inputs, is_training,data_format)

        inputs = tf.layers.conv2d(inputs,filters=filters*4, kernel_size=(1,1),strides=(1,1),padding='valid')
        inputs = batch_norm_relu(inputs, is_training,data_format)

        outputs = tf.add(inputs,short_cut,'out_put')
        return outputs
    
def block_layer(block_layer_num,blocks,inputs,filters,strides,is_training,data_format):
    block_list= ['a','b','c','d','e']
    with tf.name_scope("block_layer_%d" % block_layer_num):
        name_scope = 'conv_block_{0}_{1}'.format(block_layer_num,block_list[0])
        inputs =conv_block(inputs,filters,strides,is_training,data_format,name_scope=name_scope )
        for index in range(1,blocks):
            name_scope = 'identity_block_{0}_{1}'.format(block_layer_num,block_list[index])
            inputs = identity_block(inputs,filters,strides,is_training,data_format,name_scope)
    return inputs

In [75]:
def imagenet_resnet50_v2_generator(inputs,data_fortmat,is_training):
#     inputs = tf.placeholder(dtype=tf.float32, shape = (None,224,224,3),name = 'input_image')
    if data_format == 'channles_first':
        inputs = tf.transpose(inputs,[0,3,1,2])

    with tf.name_scope('resnet_graph'):
        #padding
        with tf.name_scope('stage_0'):
            with tf.name_scope('conv2d_fixed_padding'):
                inputs = fixed_padding(inputs,7,data_format)

                inputs = tf.layers.conv2d(inputs,filters=64,kernel_size=(7,7),strides=(2,2),data_format = data_format,
                                          padding='valid',use_bias=False,kernel_initializer=tf.variance_scaling_initializer())

            inputs = tf.identity(inputs, 'initial_conv')

            inputs = tf.layers.max_pooling2d(inputs, pool_size=(3,3),strides=(2,2),
                                            padding='same',data_format=data_format)
            inputs = tf.identity(inputs, 'initial_max_pool')


        #block_layer_1
        block_layer_num=1
        blocks=4
        filters=64
        strides=1
        inputs = block_layer(block_layer_num,blocks,inputs,filters,strides,is_training,data_format)
        print(inputs)
        
        #block_layer_2
        block_layer_num=2
        blocks=4
        filters=128
        strides=2
        inputs = block_layer(block_layer_num,blocks,inputs,filters,strides,is_training,data_format)
        print(inputs)
        
        #block_layer_3
        block_layer_num=3
        blocks=5
        filters=256
        strides=2
        inputs = block_layer(block_layer_num,blocks,inputs,filters,strides,is_training,data_format)
        print(inputs)
        
        
        #block_layer_4
        block_layer_num=4
        blocks=3
        filters=512
        strides=2
        inputs = block_layer(block_layer_num,blocks,inputs,filters,strides,is_training,data_format)
        print(inputs)
        
        
        with tf.name_scope('stage_5'):
            inputs = batch_norm_relu(inputs,is_training,data_format)
            inputs = tf.layers.average_pooling2d(inputs,pool_size=(7,7),strides = (1,1),
                                                 padding='valid',data_format=data_format)
            inputs = tf.identity(inputs, 'final_avg_pool')
            inputs = tf.reshape(inputs,shape = (-1,2048))
            
            inputs = tf.layers.dense(inputs,units=1000)
            inputs = tf.identity(inputs, 'final_dense')
            print(inputs)
    return inputs

In [87]:
class Resnet(object):
    def __init__(self,mode,data_fortmat,is_training,images,labels=None):
        self.mode = mode
        self.data_format = data_format
        
        self.is_training = is_training
        self.images = images
        self.input_shape = (None,224,224,3)
        

    
    def build_graph(self):
        self._build_model()
        if self.mode=='train':
            self._build_train_op()            
    
    def _build_model(self):
        logits = imagenet_resnet50_v2_generator(self.images,self.data_format,self.is_training)
        self.predictions = tf.argmax(logits,axis = 1)
        cross_entropy = tf.losses.softmax_cross_entropy(logits=logits,onehot_labels=labels)
        #cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,labels=labels)
        tf.identity(cross_entropy, name='cross_entropy')
        tf.summary.scalar('cross_entropy', cross_entropy)
        print(cross_entropy)
        
    
    def _build_train_op(self):
        pass
    
    def train(self):
        pass
    
    def evaluate(self):
        pass
    
    def predict(self):
        pass

In [88]:
tf.reset_default_graph()
data_format = 'channels_first' if tf.test.is_built_with_cuda() else 'channels_last'
mode = 'train'

with tf.Graph().as_default() as graph:
    with tf.name_scope('input'):
        input_shape = (None,224,224,3)
        images = tf.placeholder(dtype=tf.float32, shape = input_shape,name = 'input_images')
        labels = tf.placeholder(dtype=tf.float32, shape = (None,1000),name = 'input_labels')
        print(inputs)

    
    imagenet_resnet50= Resnet(mode,data_format,is_training,images= images,labels=labels)
    print(imagenet_resnet50)
    
    # 将images的tensor传入，作为类的参数，用作 .build_graph()建图后使用。
    # 后面run时可以用feeding模式传入数据feed_dict={images:images,labels:labels}
    # 也可以使用data pipeline 的方式直接传入到 images,labels = iterator.get_next()
    imagenet_resnet50.build_graph()
    
    
with tf.Session(graph=graph) as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    writer = tf.summary.FileWriter('log',graph)

Tensor("input/input_image:0", shape=(?, 224, 224, 3), dtype=float32)
Tensor("resnet_graph/block_layer_1/identity_block_1_d/out_put:0", shape=(?, 56, 56, 256), dtype=float32)
Tensor("resnet_graph/block_layer_2/identity_block_2_d/out_put:0", shape=(?, 28, 28, 512), dtype=float32)
Tensor("resnet_graph/block_layer_3/identity_block_3_e/out_put:0", shape=(?, 14, 14, 1024), dtype=float32)
Tensor("resnet_graph/block_layer_4/identity_block_4_c/out_put:0", shape=(?, 7, 7, 2048), dtype=float32)
Tensor("resnet_graph/stage_5/final_dense:0", shape=(?, 1000), dtype=float32)
Tensor("softmax_cross_entropy_with_logits/Reshape_2:0", shape=(?,), dtype=float32)
